# HW 3, problem 1

K-means clustering for the states crime data

In [1]:
using DataFrames, CSV
using StatsBase, Statistics
using LinearAlgebra
using Plots

US = CSV.read("USArrests.csv", DataFrame)
# murder, assault, rape per 100K
# percent urban population
first(US, 5)

,State,Murder,Assault,UrbanPop,Rape
,String15,Float64,Int64,Int64,Float64
1,Alabama,13.2,236,58,21.2
2,Alaska,10.0,263,48,44.5
3,Arizona,8.1,294,80,31.0
4,Arkansas,8.8,190,50,19.5
5,California,9.0,276,91,40.6


In [2]:
USm = Matrix(US[:,2:5]) # scaling requires a matrix

50×4 Matrix{Float64}:
 13.2  236.0  58.0  21.2
 10.0  263.0  48.0  44.5
  8.1  294.0  80.0  31.0
  8.8  190.0  50.0  19.5
  9.0  276.0  91.0  40.6
  7.9  204.0  78.0  38.7
  3.3  110.0  77.0  11.1
  5.9  238.0  72.0  15.8
 15.4  335.0  80.0  31.9
 17.4  211.0  60.0  25.8
  5.3   46.0  83.0  20.2
  2.6  120.0  54.0  14.2
 10.4  249.0  83.0  24.0
  ⋮                 
  3.4  174.0  87.0   8.3
 14.4  279.0  48.0  22.5
  3.8   86.0  45.0  12.8
 13.2  188.0  59.0  26.9
 12.7  201.0  80.0  25.5
  3.2  120.0  80.0  22.9
  2.2   48.0  32.0  11.2
  8.5  156.0  63.0  20.7
  4.0  145.0  73.0  26.2
  5.7   81.0  39.0   9.3
  2.6   53.0  66.0  10.8
  6.8  161.0  60.0  15.6

In [3]:
USsc = standardize(ZScoreTransform, USm, dims=1)
# scale each attribute to have mean 0 and std dev 1

50×4 Matrix{Float64}:
  1.24256     0.782839  -0.520907   -0.00341647
  0.507862    1.10682   -1.21176     2.4842
  0.0716334   1.4788     0.99898     1.04288
  0.232349    0.230868  -1.07359    -0.184917
  0.278268    1.26281    1.75892     2.06782
  0.0257146   0.398859   0.860809    1.86497
 -1.03042    -0.729082   0.791723   -1.08174
 -0.433474    0.806838   0.446294   -0.579946
  1.74767     1.97078    0.99898     1.13897
  2.20686     0.482855  -0.382735    0.487702
 -0.57123    -1.49704    1.20624    -0.110181
 -1.19113    -0.609088  -0.79725    -0.75077
  0.5997      0.938831   1.20624     0.295525
  ⋮                                 
 -1.00746     0.038878   1.48258    -1.38068
  1.51808     1.29881   -1.21176     0.135378
 -0.915622   -1.01707   -1.41902    -0.900241
  1.24256     0.206869  -0.451821    0.605143
  1.12777     0.362861   0.99898     0.455672
 -1.05338    -0.609088   0.99898     0.178084
 -1.28297    -1.47304   -2.31714    -1.07106
  0.163471   -0.177111  -0.17

In [4]:
# a function to compute the squared euclidean distance between
# two vectors x and y.
function distance(x, y)
    return norm(x-y)^2
end

distance (generic function with 1 method)

In [5]:
# a function to compute the centroid of a cluster.
# X is a matrix that contains the cluster,
# with rows as observations and (numeric) columns as attributes.
function centroid(X)
    return mean(X, dims=1)
end

centroid (generic function with 1 method)

In [6]:
# a single iteration of kmeans
function kmeans1(X, k)
    (m,n) = size(X)
    if k > m
        error("number of clusters ", k, " > number of observations ", m)
    end

    clusters0 = rand(1:k, m)  # random initial assignment
    clusters = zeros(Int8, m) # will hold the cluster assignments
    
    ii = 0
    while true
        # compute the cluster centroids
        c = zeros(k, n)
        for i in 1:k
            c[i,:] = centroid( X[ findall(x->x==i, clusters0), : ] )
        end

        # assign each observation to the nearest centroid
        for i in 1:m
            clusters[i] = 1  # initially assign observation to cluster 1
            best = distance(X[i,:], c[1,:])
            for j in 2:k
                candidate = distance(X[i,:], c[j,:])
                if candidate < best  # assign to cluster j if closer
                    best = candidate
                    clusters[i] = j
                end
            end
        end
        if clusters == clusters0
            break
        end
        clusters0 = clusters
        ii += 1
        if ii % 1 == 0
            println("iteration ", ii)
        end
    end
    ncl = length(countmap(clusters))
    if ncl != k
        @warn "clustering solution contains "*string(ncl)*" < "*string(k)*" clusters."
    end
    return clusters
end

kmeans1 (generic function with 1 method)

In [7]:
# a function compute the value of the kmeans objective
# function, the sum of the within-cluster distances.
# X is the matrix of observations.
# k is the number of clusters.
# cl is the clustering solution.
function objective(X, k, cl)
    result = 0

    # compute the cluster centroids
    c = zeros(k, size(X,2))
    for i in 1:k
        c[i,:] = centroid( X[ findall(x->x==i, cl), : ] )
    end
    
    # compute the distances within each cluster and
    # add to the total result
    for i in 1:k
        X_cl = X[findall(x->x==i, cl), :]
        for j in 1:size(X_cl, 1)
            result += norm(X_cl[j, :] - c[i, :])^2
        end
    end
    
    return result
end

objective (generic function with 1 method)

In [8]:
# driver function for kmeans.
# X is the (scaled) matrix of observations.
# k is the number of clusters.
# niter is the number of times to run the k-means algorithm.
# the best of the niter candidate solutions is returned.
function kmeans(X, k; niter=50)
    
    clusters = kmeans1(X, k)
    bestCl = clusters
    bestVal = objective(X, k, clusters)
    
    for n in 1:niter
        clusters = kmeans1(X, k)
        val = objective(X, k, clusters)
        println("candidate solution ", n, ": ", val)
        if val < bestVal
            bestVal = val
            bestCl = clusters
        end
    end
    
    return bestCl
end

kmeans (generic function with 1 method)

In [9]:
cl = kmeans(USsc, 4);    # call kmeans driver function

iteration 1
iteration 1
candidate solution 1: 71.25993059839846
iteration 1
candidate solution 2: 76.40636380117655
iteration 1
candidate solution 3: 67.71944137992175
iteration 1
candidate solution 4: 77.55998115147533
iteration 1
candidate solution 5: 78.15529285980382
iteration 1
candidate solution 6: 70.78549386515611
iteration 1
candidate solution 7: 81.07039163465508
iteration 1
candidate solution 8: 63.794494847734526
iteration 1
candidate solution 9: 62.69857041763498
iteration 1
candidate solution 10: 77.0939945548463
iteration 1
candidate solution 11: 70.58537814891393
iteration 1
candidate solution 12: 60.194325489254375
iteration 1
candidate solution 13: 75.85090134403768
iteration 1
candidate solution 14: 56.96390255050683
iteration 1
candidate solution 15: 70.22999472735529
iteration 1
candidate solution 16: 70.11491715046824
iteration 1
candidate solution 17: 60.68820608213485
iteration 1
candidate solution 18: 77.01960633866868
iteration 1
candidate solution 19: 68.8252

In [10]:
objective(USsc, 4, cl)   # the objective function of the best solution found

56.96390255050683

In [11]:
cldict = countmap(cl)    # use countmap() to see the number of obs in each cluster

Dict{Int8, Int64} with 4 entries:
  4 => 15
  2 => 8
  3 => 14
  1 => 13

In [12]:
# obtain information from the clustering to solution to provide
# a qualitative description of each of the four clusters of states.
vars = ["murder", "assault", "urbanpop", "rape"]

for i in 1:4
    for j in keys(cldict)
        c = centroid(USm[ findall(x->x==i, cl), : ])
        println("mean of cluster ", i, " for ", vars[j], " = ", round(c[j], digits=2))
    end
    println()
end

mean of cluster 1 for rape = 33.19
mean of cluster 1 for assault = 257.38
mean of cluster 1 for urbanpop = 76.0
mean of cluster 1 for murder = 10.82

mean of cluster 2 for rape = 21.41
mean of cluster 2 for assault = 243.62
mean of cluster 2 for urbanpop = 53.75
mean of cluster 2 for murder = 13.94

mean of cluster 3 for rape = 12.42
mean of cluster 3 for assault = 84.43
mean of cluster 3 for urbanpop = 52.64
mean of cluster 3 for murder = 3.83

mean of cluster 4 for rape = 18.99
mean of cluster 4 for assault = 137.4
mean of cluster 4 for urbanpop = 74.8
mean of cluster 4 for murder = 5.58



Ranking (lowest to highest) <br>
rape: 3, 4, 2, 1 <br>
assault: 3, 4, 2, 1 <br>
urbanpop: 3, 2, 4, 1 <br>
murder: 3, 4, 1, 2

States in cluster 1 have the highest mean percent urban population, highest number of rape arrests as well as assault arrests. States in cluster 3 score lowest on all 4 attributes: lowest percent urban population, and lowest number of arrests for all 3 crimes. Compared to those in cluster 2, states in cluster 4 have a higher percent urban population but lower number of arrests for all 3 crimes. States in cluster 2 have the highest number of murder arrests.